<a href="https://colab.research.google.com/github/okj3031402/LLM_study/blob/main/building_composite_function_calling_chatbot_app_with_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import requests
import json

def get_current_location():
    response = requests.get('https://ipinfo.io')
    info = response.json()
    return json.dumps(info)

In [ ]:
import requests

def get_current_weather(latitude, longitude):

    # Define the base URL for the Open-Meteo API
    base_url = "https://api.open-meteo.com/v1/forecast"

    # Specify your parameters (in this case, for Berlin, Germany)
    params = {
        "latitude": latitude,   # Example latitude
        "longitude": longitude,  # Example longitude
        "current_weather": True, # To get current weather
        "daily": "temperature_2m_max,temperature_2m_min", # For daily max/min temperatures
        "timezone": "auto" # Automatically determine the timezone
    }

    # Make the GET request
    response = requests.get(base_url, params=params)

    info = {}
    # Check if the request was successful
    if response.status_code == 200:
        info = response.json()

    return json.dumps(info)

In [ ]:
!pip install yfinance

import yfinance as yf
import json

def get_stock_price_info(stock_symbol):
    # yfinance를 사용하여 주식 정보 로드

    print(stock_symbol)

    stock = yf.Ticker(stock_symbol)

    # 주식의 현재 정보 가져오기
    info = stock.info

    # 관심 있는 정보를 선택적으로 추출
    relevant_info = {
        'currentPrice': info.get('currentPrice'),
        'previousClose': info.get('previousClose'),
        'open': info.get('open'),
        'dayHigh': info.get('dayHigh'),
        'dayLow': info.get('dayLow'),
        'volume': info.get('volume'),
        'marketCap': info.get('marketCap')
    }

    # JSON 형태로 정보 반환
    return json.dumps(relevant_info, ensure_ascii=False)


In [ ]:
functions = [
    {
        "name": "get_current_location",
        "description": "Get the current location like latitude and longitude.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "get_current_weather",
        "description": "Get the current weather like temperature and weather type in a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "string",
                    "description": "latitude",
                },
                "longitude": {
                    "type": "string",
                    "description": "longitude",
                },
            },
            "required": ["latitude", "longitude"],
        }
    },
    {
        "name": "get_stock_price_info",
        "description": "Retrieves the stock price information for a specific stock using Yahoo Finance API. For Korean stocks, append '.KS' for KOSPI listed stocks and '.KQ' for KOSDAQ listed stocks to the stock symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "stock_symbol": {
                    "type": "string",
                    "description": "The symbol of the stock to retrieve information for. For Korean stocks, use the stock code followed by '.KS' for KOSPI stocks or '.KQ' for KOSDAQ stocks, e.g., '005930.KS' for Samsung Electronics."
                }
            },
            "required": ["stock_symbol"]
        }
    }
]

In [ ]:
toolkits = {
    "get_current_location": get_current_location,
    "get_current_weather": get_current_weather,
    "get_stock_price_info": get_stock_price_info,
}

In [ ]:
system_prompt = """
Act as helpful assistant. Write in Korean.
To handle user requests, we determine which function to call first and try to acquire the necessary information from the given list of functions as much as possible.
"""

def print_message(message):
    print(json.dumps(message, indent=4))
    #if message.get("function_call"):
    #    print(message['function_call']['name'])

def openai_llm(input_text, chat_history):

    if len(chat_history) == 0:
        chat_history.append({"role": "system", "content": system_prompt})

    # input_text : 안녕
    chat_history.append({"role": "user", "content": input_text})

    while True: # 함수 호출을 더 이상하지 않을 때까지 루프

        response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                                messages=chat_history,
                                                functions=functions,
                                                function_call="auto")

        response_message = response["choices"][0]["message"]

        # AI의 응답을 추가
        chat_history.append(response_message)

        print_message(response_message) # 주석처리해도 됩니다.

        # function_call의 조건
        # 1. 함수 호출을 하지 못할 때, 필요한 인자가 메시지의 리스트에 없을 경우 (편지봉투 내에) > 사용자에게 함수 인자에 대한 정보를 요청
        # 2. 함수 호출을 할 필요가 없을 때 > 함수 호출 결과를 요약

        # response_message에는 role, content, function_call가 있다.
        # function_call 있다면, content = null
        # content에 값이 있다면, function_call 키가 존재하지 않는다.
        if response_message.get("function_call") == None:
            break

        # 함수 이름과 인자 값을 얻어온다.
        function_name = response_message["function_call"]["name"]
        function_arguments = json.loads(response_message["function_call"]["arguments"])

        # 함수를 호출해서, 그 결과를 얻어온다.
        function_response = toolkits[function_name](**function_arguments)

        function_response_message = {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }

        # 함수 호출 결과를 추가한다.
        chat_history.append(function_response_message)

        print_message(function_response_message) # 주석처리해도 됩니다.

    output = response.choices[0].message.content # function_call 키가 있다면, content는 null이나, function_call 키가 없을 때, break되므로 항상 content가 있다.

    return output

In [ ]:
print(openai_llm("지금 현재 위치 날씨는?", []))

{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_location",
        "arguments": "{}"
    }
}
{
    "role": "function",
    "name": "get_current_location",
    "content": "{\"ip\": \"34.85.190.148\", \"hostname\": \"148.190.85.34.bc.googleusercontent.com\", \"city\": \"Washington\", \"region\": \"Washington, D.C.\", \"country\": \"US\", \"loc\": \"38.8951,-77.0364\", \"org\": \"AS396982 Google LLC\", \"postal\": \"20004\", \"timezone\": \"America/New_York\", \"readme\": \"https://ipinfo.io/missingauth\"}"
}
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_weather",
        "arguments": "{\n  \"latitude\": \"38.8951\",\n  \"longitude\": \"-77.0364\"\n}"
    }
}
{
    "role": "function",
    "name": "get_current_weather",
    "content": "{\"latitude\": 38.890526, \"longitude\": -77.02716, \"generationtime_ms\": 0.0699758529663086, \"utc_offset_seconds\": -14400, \"timezone\": \"America/N

In [ ]:
print(openai_llm("삼성전자 주가는?", []))

{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_stock_price_info",
        "arguments": "{\n  \"stock_symbol\": \"005930.KS\"\n}"
    }
}
005930.KS
{
    "role": "function",
    "name": "get_stock_price_info",
    "content": "{\"currentPrice\": 78500.0, \"previousClose\": 78200.0, \"open\": 78600.0, \"dayHigh\": 78800.0, \"dayLow\": 78200.0, \"volume\": 2718321, \"marketCap\": 522072904171520}"
}
{
    "role": "assistant",
    "content": "\uc0bc\uc131\uc804\uc790\uc758 \ud604\uc7ac \uc8fc\uac00\ub294 78,500\uc6d0\uc785\ub2c8\ub2e4. \uc804\uc77c \uc885\uac00\ub294 78,200\uc6d0\uc774\uba70, \uc2dc\uac00\ub294 78,600\uc6d0\uc785\ub2c8\ub2e4. \ucd5c\uace0\uac00\ub294 78,800\uc6d0\uc774\uace0, \ucd5c\uc800\uac00\ub294 78,200\uc6d0\uc785\ub2c8\ub2e4. \uac70\ub798\ub7c9\uc740 2,718,321\uc8fc\uc774\uba70, \uc2dc\uac00\ucd1d\uc561\uc740 522,072,904,171,520\uc6d0\uc785\ub2c8\ub2e4."
}
삼성전자의 현재 주가는 78,500원입니다. 전일 종가는 78,200원이며, 시가는 78,600원입니다. 최고가는 78,8

In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = openai_llm(user_message, chat_history)
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > 서울의 날씨는?
{
    "role": "assistant",
    "content": null,
    "function_call": {
        "name": "get_current_weather",
        "arguments": "{\n  \"latitude\": \"37.5665\",\n  \"longitude\": \"126.9780\"\n}"
    }
}
{
    "role": "function",
    "name": "get_current_weather",
    "content": "{\"latitude\": 37.55, \"longitude\": 127.0, \"generationtime_ms\": 0.08296966552734375, \"utc_offset_seconds\": 32400, \"timezone\": \"Asia/Seoul\", \"timezone_abbreviation\": \"KST\", \"elevation\": 34.0, \"current_weather_units\": {\"time\": \"iso8601\", \"interval\": \"seconds\", \"temperature\": \"\\u00b0C\", \"windspeed\": \"km/h\", \"winddirection\": \"\\u00b0\", \"is_day\": \"\", \"weathercode\": \"wmo code\"}, \"current_weather\": {\"time\": \"2024-05-17T09:30\", \"interval\": 900, \"temperature\": 17.0, \"windspeed\": 3.6, \"winddirection\": 174, \"is_day\": 1, \"weathercode\": 1}, \"daily_units\": {\"time\": \"iso8601\", \"temperature_2m_max\": \"\\u00b0C\", \"temperature_2m_min\":